# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.10574


extracting tarball to tmp_2212.10574...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.10576


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.10574/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2212.11096...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.11177


extracting tarball to tmp_2212.11177...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.11177/pahmaps.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'authors' from 'tmp_2212.11177/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10804-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10804) | **Local Group Dwarf Galaxy Detection Limit in the CSST survey**  |
|| Han Qu, et al. -- incl., <mark>Guoliang Li</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *submitted to MNRAS. 11 pages, 10 figures*|
|**Abstract**| We predict the dwarf galaxy detection limits for the upcoming Chinese Space Station Telescope (CSST) survey that will cover 17,500 deg$^{2}$ of the sky with a wide field of view of 1.1 deg$^2$. The point-source depth reaches 26.3 mag in the $g$ band and 25.9 mag in the $i$ band. Constructing mock survey data based on the designed photometric bands, we estimate the recovery rate of artificial dwarf galaxies from mock point-source photometric catalogues. The detection of these artificial dwarf galaxies is strongly dependent on their distance, magnitude and size, in agreement with searches in current surveys. We expect CSST to enable the detection of dwarf galaxies with $M_V = -3.0$ and $\mu_{250} = 32.0$ mag/arcsec$^2$ (surface-brightness limit for a system of half-light radius $r_{\rm h}$ = 250 pc at 400 kpc, and $M_V = -4.9$ and $\mu_{250} = 30.5$ mag/arcsec$^2$ around the Andromeda galaxy. Beyond the Local Group, the CSST survey will achieve $M_V = -5.8$, and $\mu_{250}$ = 29.7 mag/arcsec$^2$ in the distance range of 1--2 Mpc, opening up an exciting discovery space for faint field dwarf galaxies. With its optical bands, wide survey footprint, and space resolution, CSST will undoubtedly expand our knowledge of low-mass dwarf galaxies to an unprecedented volume. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11177-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11177) | **PHANGS-JWST First Results: Tracing the Diffuse ISM with JWST Imaging of  Polycyclic Aromatic Hydrocarbon Emission in Nearby Galaxies**  |
|| Karin M. Sandstrom, et al. -- incl., <mark>Rowan J. Smith</mark>, <mark>Janice C. Lee</mark>, <mark>Eva Schinnerer</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *18 pages, 6 figures, accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| JWST observations of polycyclic aromatic hydrocarbon (PAH) emission provide some of the deepest and highest resolution views of the cold interstellar medium (ISM) in nearby galaxies. If PAHs are well mixed with the atomic and molecular gas and illuminated by the average diffuse interstellar radiation field, PAH emission may provide an approximately linear, high resolution, high sensitivity tracer of diffuse gas surface density. We present a pilot study that explores using PAH emission in this way based on MIRI observations of IC 5332, NGC 628, NGC 1365, and NGC 7496 from the PHANGS-JWST Treasury. Using scaling relationships calibrated in Leroy et al. (2022), scaled F1130W provides 10--40 pc resolution and 3$\sigma$ sensitivity of $\Sigma_{\rm gas} \sim 2$ M$_\odot$ pc$^{-2}$. We characterize the surface densities of structures seen at $< 7$ M$_\odot$ pc$^{-2}$ in our targets, where we expect the gas to be HI-dominated. We highlight the existence of filaments, inter-arm emission, and holes in the diffuse ISM at these low surface densities. Below $\sim 10$ M$_\odot$ pc$^{-2}$ for NGC 628, NGC 1365, and NGC 7496 the gas distribution shows a ``Swiss cheese''-like topology due to holes and bubbles pervading the relatively smooth distribution of diffuse ISM. Comparing to recent galaxy simulations, we observe similar topology for the low surface density gas, though with notable variations between simulations with different setups and resolution. Such a comparison of high resolution, low surface density gas with simulations is not possible with existing atomic and molecular gas maps, highlighting the unique power of JWST maps of PAH emission. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10574-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10574) | **PHANGS-JWST First Results: Mid-infrared emission traces both gas column  density and heating at 100 pc scales**  |
|| Adam K. Leroy, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Janice C. Lee</mark>, <mark>Daizhong Liu</mark>, <mark>Justus Neumann</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *49 pages, 17 figures, Section 8 provides a detailed summary, submitted to AAS journals and revised according to referee comments, part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We compare mid-infrared (mid-IR), extinction-corrected H$\alpha$, and CO (2-1) emission at 70--160 pc resolution in the first four PHANGS-JWST targets. We report correlation strengths, intensity ratios, and power law fits relating emission in JWST's F770W, F1000W, F1130W, and F2100W bands to CO and H$\alpha$. At these scales, CO and H$\alpha$ each correlate strongly with mid-IR emission, and these correlations are each stronger than the one relating CO to H$\alpha$ emission. This reflects that mid-IR emission simultaneously acts as a dust column density tracer, leading to the good match with the molecular gas-tracing CO, and as a heating tracer, leading to the good match with the H$\alpha$. By combining mid-IR, CO, and H$\alpha$ at scales where the overall correlation between cold gas and star formation begins to break down, we are able to separate these two effects. We model the mid-IR above $I_\nu = 0.5$~MJy sr$^{-1}$ at F770W, a cut designed to select regions where the molecular gas dominates the interstellar medium (ISM) mass. This bright emission can be described to first order by a model that combines a CO-tracing component and an H$\alpha$-tracing component. The best-fitting models imply that $\sim 50\%$ of the mid-IR flux arises from molecular gas heated by the diffuse interstellar radiation field, with the remaining $\sim 50\%$ associated with bright, dusty star forming regions. We discuss differences between the F770W, F1000W, F1130W bands and the continuum dominated F2100W band and suggest next steps for using the mid-IR as an ISM tracer. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11028-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11028) | **Effects of Multiphase Gas and Projection on X-ray Observables in  Simulated Galaxy Clusters as Seen by eROSITA**  |
|| J. ZuHone, et al. -- incl., <mark>T. Liu</mark>, <mark>O. König</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *17 pages, 16 figures, submitted to A&A*|
|**Abstract**| The number density of galaxy clusters as a function of mass and redshift is a sensitive function of the cosmological parameters. To use clusters for cosmological parameter studies, it is necessary to determine their masses as accurately as possible, which is typically done via mass-observable scaling relations. X-ray observables can be biased by multiphase gas and projection effects, especially in the case where cluster temperatures and luminosities are estimated from single-model fits to all of the emission with a given radius. Using simulated galaxy clusters from a realistic cosmological simulation, we seek to determine the importance of these biases in the context of Spectrum-Roentgen-Gamma/eROSITA observations of clusters. We extract clusters from the Magneticum suite, and simulate eROSITA observations of these clusters using PHOX and SIXTE. We compare the fitted observables from these observations to those derived from the simulations. We fitted an intrinsically scattered $L_{\rm X}-T$ scaling relation to these measurements following a Bayesian approach with which we fully took into account the selection effects and the mass function. The largest biases on the cluster observables come from the inadequacy of single-temperature model fits to represent emission from multiphase gas, as well as a bias arising from cluster emission within the projected $r_{500c}$ along the line of sight but outside of the spherical $r_{500c}$. We find that the biases on temperature and luminosity due to the projection of emission from other clusters within $r_{500c}$ is small. We find that our simulated clusters follow a $L_{\rm X}-T$ scaling relation that has a broadly consistent but slightly shallower slope compared to the literature, and that the intrinsic scatter of $L_{\rm X}$ at given T is lower compared to the recent observational results where the selection effects are fully considered. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11034-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11034) | **Hunting for Neighboring Open Clusters with Gaia DR3: 101 New Open  Clusters within 500 pc**  |
|| Songmei Qin, Jing Zhong, Tong Tang, <mark>Li Chen</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *32 pages, 10 figures*|
|**Abstract**| We systematically searched for open clusters in the solar neighborhood within 500 pc using pyUPMASK and HDBSCAN clustering algorithms based on {\it Gaia} DR3. Taking into consideration that the physical size for most open clusters is less than 50 pc, we adopted a slicing approach for different distance shells and identified 324 neighboring open clusters, including 223 reported open clusters and 101 newly discovered open clusters (named as OCSN, Open Cluster of Solar Neighborhood). Our discovery has increased the number of open clusters in the solar neighborhood by about 45\%. In this work, larger spatial extents and more member stars were attained for our cluster sample. We provided the member stars and the membership probabilities through the pyUPMASK algorithm for each cluster and derived their astrophysical, age, and structural parameters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11044-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11044) | **Oscillations and tidal deformations of crystallized white dwarfs**  |
|| Yat-To Tang, <mark>Lap-Ming Lin</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *12 pages, 18 figures*|
|**Abstract**| Long predicted more than fifty years ago, strong evidence for the existence of crystalline cores inside white dwarfs has recently been obtained by the Gaia space telescope. It is thus important to investigate how a crystalline core may affect the properties and dynamics of white dwarfs. In this paper, we first study the dependence of the frequencies of the fundamental (f), interfacial (i), and shear (s) oscillation modes on the size of the crystalline core. We find that the frequencies of the i- and s-modes depend sensitively on the size of the core, while the frequency of the f-mode is affected only slightly by at most a few percent for our chosen white dwarf models. We next consider the tidal deformability of crystallized white dwarfs and find that the effect of crystallization becomes significant only when the radius of the core is larger than about 70% of the stellar radius. The tidal deformability can change by a few to about 10 percent when a white dwarf becomes fully crystallized. We also show that there exist approximate equation-of-state insensitive relations connecting the mass, moment of inertia, tidal deformability, and f-mode frequency for pure fluid white dwarfs. Depending on the stellar mass and composition, however, these relations can be affected by a few percent when the white dwarf is crystallized. These changes could leave an imprint on the gravitational waves emitted from the late inspiral or merger of white dwarf binaries, which may be detectable by future space-borne gravitational wave detectors. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11096-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11096) | **The Sun at millimeter wavelengths IV. Magnetohydrodynamic waves in  small-scale bright features**  |
|| Juan Camilo Guevara Gómez, et al. -- incl., <mark>Henrik Eklund</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *15 pages, 11 figures. Accepted for publication in Astronomy and Astrophysics. Abstract has been shorten to fulfil Arxiv requirements*|
|**Abstract**| We used solar observations of a plage/enhanced network with the Atacama Large Millimeter/sub-millimeter Array (ALMA) in Band 3 and Band 6 together with synthetic continuum maps from numerical simulations with Bifrost at the same bands to carry out a detailed study of bright small-scale magnetic features. To this end, we have used an algorithm to automatically identify and trace the features within the field of view (FoV) of the observations and the simulation. We found 193 and 293 features in the Bands 3 and 6 observations, respectively. In the degraded simulation, the total number of features were 24 for Band 3 and 204 for Band 6. In the original simulation, the total number of features were 36 for Band 3 and 392 for Band 6. Based on the simulation, we confirm the magnetic nature of the features which exhibit an oscillatory behaviour in temperature, size and horizontal velocity. The average oscillation periods were of 30-99\,s for temperature, 37-92\,s for size and 37-78\,s for horizontal velocity. There are indications for the possible presence of transverse (kink) waves with average amplitude velocities of 2.1-5.0\,km\,s$^{-1}$. An anti-phase behaviour between temperature and size oscillations suggest the presence of compressible fast-sausage Magnetohydrodynamics (MHD) modes. Finally, we have estimated the flux of energy of the fast-sausage waves at the chromospheric heights sampled by ALMA as 453-1838\,W\,m$^{-2}$ for Band 3 and 3640-5485\,W\,m$^{-2}$ for Band 6. The decrease of wave energy-flux with height (from Band 6 to Band 3) could possibly suggest energy dissipation at chromospheric heights, thus wave heating, with the assumptions that the identified small-scale waves are typical at each band and they propagate upward through the chromosphere. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10576-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10576) | **Constraints on strong phase transitions in neutron stars**  |
|| Tyler Gorda, et al. -- incl., <mark>Kai Hebeler</mark> |
|*Appeared on*| *2022-12-22*|
|*Comments*| *10 pages, 10 figures*|
|**Abstract**| We study current bounds on strong first-order phase transitions (PTs) along the equation of state (EOS) of dense strongly interacting matter in neutron stars, under the simplifying assumption that on either side of the PT the EOS can be approximated by a simple polytropic form. We construct a large ensemble of possible EOSs of this form, anchor them to chiral effective field theory calculations at nuclear density and perturbative QCD at high densities, and subject them to astrophysical constraints from high-mass pulsars and gravitational-wave observations. Within this setup, we find that a PT permits neutron-star solutions with larger radii, but only if the transition begins below twice nuclear saturation density. We also identify a large parameter space of allowed PTs currently unexplored by numerical-relativity studies. Additionally, we locate a small region of parameter space allowing twin-star solutions, though we find them to only marginally pass the current astrophysical constraints. Finally, we find that sizeable cores of high-density matter beyond the PT may be located in the centers of some stable neutron stars, primarily those with larger masses. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure apj_figures/ep_fig_CAUSAL_PRX.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.10804.md
    + _build/html/tmp_2212.10804/./nfig/limit_ru_m.png
    + _build/html/tmp_2212.10804/./nfig/m1_detect3.png
    + _build/html/tmp_2212.10804/./nfig/comp_uncert.png
exported in  _build/html/2212.11177.md
    + _build/html/tmp_2212.11177/./figures/ngc7496_crop.png
    + _build/html/tmp_2212.11177/./figures/ic5332_crop.png
    + _build/html/tmp_2212.11177/./figures/genus_figure_with_flair.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\gkai}[1]{\begin{CJK*}{UTF8}{gkai}\raisebox{.1em}{(}#1\raisebox{.1em}{)}\end{CJK*}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\eg}{{ e.g., }}$
$\newcommand{\ie}{{ i.e., }}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degg}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\Mpc}{{\rm Mpc}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\degx}{{\rm deg}}$
$\newcommand{\msun}{{\rm M_{\odot}}}$
$\newcommand{\FeH}{{\rm[Fe/H]}}$
$\newcommand{\}{zy}$
$\newcommand{\}{NFM}$
$\newcommand{\}{Han}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\gkai$}[1]{\begin{CJK*}{UTF8}{gkai}\raisebox{.1em}{(}#1\raisebox{.1em}{)}\end{CJK*}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\eg$}{{ e.g., }}$
$\newcommand{$\ie$}{{ i.e., }}$
$\newcommand{$\deg$}{^\circ}$
$\newcommand{$\deg$g}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{$\kpc$}{{\rm kpc}}$
$\newcommand{$\Mpc$}{{\rm Mpc}}$
$\newcommand{$\pc$}{{\rm pc}}$
$\newcommand{$\deg$x}{{\rm deg}}$
$\newcommand{$\msun$}{{\rm M_{\odot}}}$
$\newcommand{$\FeH$}{{\rm[Fe/H]}}$
$\newcommand{\}{zy}$
$\newcommand{\}{NFM}$
$\newcommand{\}{Han}$</div>



<div id="title">

# Local Group Dwarf Galaxy Detection Limit in the CSST survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.10804-b31b1b.svg)](https://arxiv.org/abs/2212.10804)<mark>Appeared on: 2022-12-22</mark> - _submitted to MNRAS. 11 pages, 10 figures_

</div>
<div id="authors">

Han Qu, et al. -- incl., <mark><mark>Guoliang Li</mark></mark>

</div>
<div id="abstract">

**Abstract:** We predict the dwarf galaxy detection limits for the upcoming Chinese Space Station Telescope (CSST) survey that will cover 17,500$\degx^{2}$of the sky with a wide field of view of 1.1 deg$^2$. The point-source depth reaches 26.3 mag in the$g$band and 25.9 mag in the$i$band. Constructing mock survey data based on the designed photometric bands, we estimate the recovery rate of artificial dwarf galaxies from mock point-source photometric catalogues. The detection of these artificial dwarf galaxies is strongly dependent on their distance, magnitude and size, in agreement with searches in current surveys. We expect CSST to enable the detection of dwarf galaxies with$M_V = -3.0$and$\mu_{250} = 32.0$mag/arcsec$^2$(surface-brightness limit for a system of half-light radius$r_{\rm h}$= 250$\pc$) at$400 $\kpc$$,  and$M_V = -4.9$and$\mu_{250} = 30.5$mag/arcsec$^2$around the Andromeda galaxy. Beyond the Local Group, the CSST survey will achieve$M_V = -5.8$, and$\mu_{250}$= 29.7 mag/arcsec$^2$in the distance range of 1--2 Mpc, opening up an exciting discovery space for faint field dwarf galaxies. With its optical bands, wide survey footprint, and space resolution, CSST will undoubtedly expand our knowledge of low-mass dwarf galaxies to an unprecedented volume.

</div>

<div id="div_fig1">

<img src="tmp_2212.10804/./nfig/limit_ru_m.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparisons of detection limit as a function of $M_V$ and $r_{\rm h}$ in different distance bins for a M field. Same as Fig. \ref{detect_m}, the error bars represent $\sigma_{M}$ and $\sigma_{r}$ respectively in the left panel, based on that the error bar of $\mu$ is derived and shown in the right panel. The dashed and solid lines represent the results with single and double exposures respectively.
    (*detect limit*)

</div>
<div id="div_fig2">

<img src="tmp_2212.10804/./nfig/m1_detect3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Detection fraction map as a function of luminosity and size in different distance bins for M field, where the black pixels indicates 100\% detection fraction, and the white ones indicates 0\%. The red curve corresponds to the modeled  50\% recovery fraction and the error bars represent $\sigma_{M}$ and $\sigma_{r}$ as the widths of the transition regions.
    (*detect_m*)

</div>
<div id="div_fig3">

<img src="tmp_2212.10804/./nfig/comp_uncert.png" alt="Fig2" width="100%"/>

**Figure 2. -** \emph{Left:} Completeness model for point-source detections as a function of $g$(blue) and $i$(red) magnitude estimates for the CSST. The solid and dashed lines represent the single- and double-exposure modes respectively. \emph{Right:} Expected photometric uncertainties in the two bands for the one-exposure model (crosses) and fitted model (dashed lines). The two-exposure uncertainty model (full lines) corresponds to a shift of these models by 0.38 magnitudes to account for the increased exposure time.
     (*fig:comp_uncert*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mjysr}{MJy sr^{-1}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msolpcsq}{M_\odot pc^{-2}}$
$\newcommand{\hi}{\mbox{\sc{Hi} }}$
$\newcommand{\htwo}{H_{2}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\mjysr$}{MJy sr^{-1}}$
$\newcommand{$\msun$}{M_{\odot}}$
$\newcommand{$\msolpcsq$}{M_\odot pc^{-2}}$
$\newcommand{$\hi$}{\mbox{\sc{Hi} }}$
$\newcommand{$\htwo$}{H_{2}}$</div>



<div id="title">

# PHANGS-JWST First Results: Tracing the Diffuse ISM with JWST Imaging of Polycyclic Aromatic Hydrocarbon Emission in Nearby Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.11177-b31b1b.svg)](https://arxiv.org/abs/2212.11177)<mark>Appeared on: 2022-12-22</mark> - _18 pages, 6 figures, accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

Karin M. Sandstrom, et al. -- incl., <mark>Rowan J. Smith</mark>, <mark>Janice C. Lee</mark>, <mark>Eva Schinnerer</mark>, <mark>Jonathan~D.~Henshaw</mark>, <mark>Annie~Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>

</div>
<div id="abstract">

**Abstract:** JWST observations of polycyclic aromatic hydrocarbon (PAH) emission provide some of the deepest and highest resolution views of the cold interstellar medium (ISM) in nearby galaxies. If PAHs are well mixed with the atomic and molecular gas and illuminated by the average diffuse interstellar radiation field, PAH emission may provide an approximately linear, high resolution, high sensitivity tracer of diffuse gas surface density. We present a pilot study that explores using PAH emission in this way based on MIRI observations of IC 5332, NGC 628, NGC 1365, and NGC 7496 from the PHANGS-JWST Treasury. Using scaling relationships calibrated in\citet{LEROY1_PHANGSJWST}, scaled F1130W provides 10--40 pc resolution and 3$\sigma$sensitivity of$\Sigma_{\rm gas} \sim 2$\msunpc$^{-2}$. We characterize the surface densities of structures seen at$<7$\msunpc$^{-2}$in our targets, where we expect the gas to be HI-dominated. We highlight the existence of filaments, inter-arm emission, and holes in the diffuse ISM at these low surface densities. Below$\sim 10$\msunpc$^{-2}$for NGC 628, NGC 1365, and NGC 7496 the gas distribution shows a "Swiss cheese’’-like topology due to holes and bubbles pervading the relatively smooth distribution of diffuse ISM. Comparing to recent galaxy simulations, we observe similar topology for the low surface density gas, though with notable variations between simulations with different setups and resolution. Such a comparison of high resolution, low surface density gas with simulations is not possible with existing atomic and molecular gas maps, highlighting the unique power of JWST maps of PAH emission.

</div>

<div id="div_fig1">

<img src="tmp_2212.11177/./figures/ngc7496_crop.png" alt="Fig5" width="100%"/>

**Figure 5. -** NGC 7496 gas surface density from scaled F1130W emission. A single black contour highlights the brightness level where the F1130W emission implies $\Sigma_{\rm gas} = 7$\msun pc$^{-2}$. In NGC 7496, saturation in the center leads to an artifact where the columns of pixels with saturation show low values. This is evident in the faint regions of our map. (*fig:contour_7496*)

</div>
<div id="div_fig2">

<img src="tmp_2212.11177/./figures/ic5332_crop.png" alt="Fig2" width="100%"/>

**Figure 2. -** IC 5332 gas surface density from scaled F1130W emission. A single black contour highlights the brightness level where the F1130W emission implies $\Sigma_{\rm gas} = 7$\msun pc$^{-2}$. This figure and the subsequent figures for NGC 628, 1365, and 7496 (Figures \ref{fig:contour_ngc628},  \ref{fig:contour_1365}, and  \ref{fig:contour_7496}) are all shown with the same color bar. We show a 500 pc scale bar for reference. (*fig:contour_ic5332*)

</div>
<div id="div_fig3">

<img src="tmp_2212.11177/./figures/genus_figure_with_flair.png" alt="Fig1" width="100%"/>

**Figure 1. -** Genus curves for the F1130W band, scaled to gas surface density (Equation \ref{eq:ratio113}).
The gray region shows where the noise dominates the statistic up to $3\sigma$ significance above the background which corresponds to $\Sigma_{\rm gas} \approx 2$ $\msolpcsq$.
For a consistent comparison, we impose a radial cut at $0.4 R_{25}$ in each galaxy, corresponding to 3.2, 5.7, 13.7, 3.6 kpc in IC 5332, NGC 628, NGC 1365 and NGC 7496, respectively.
The $11.3$ $\mu$m emission traces a clear conversion from a "Swiss cheese" (Genus$<0$) to "meatball" topology (Genus$>0$) when going from low to high gas surface density.
 (*fig:genus_1130*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

145  publications in the last 7 days.
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers